# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import sys
sys.path.append('C:/Users/User/Desktop/20Newsgroups')
from data_preprocessing import *
from transformer_build import  *

# 데이터 로드

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

# 전처리

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

df_newsgroup['target'] = le.transform(df_newsgroup['target'])

X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

x_train=list(X_train)
x_test=list(X_test)

sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos>'+sen)
for sen in x_test:
    sos_x_test.append('<sos>'+sen)

all_txt=sos_x_train+sos_x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [4]:
y_train = np.array(y_train)
y_test = np.array(y_test)

x_val=x_train[:1000]
x_train=x_train[1000:]
y_val=y_train[:1000]
y_train=y_train[1000:]

# 모델 빌드

In [5]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [6]:
embedding_dim = 256  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    x = transformer_block(x)

# transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
# x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(20, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 token_and_position_embeddin  (None, 300, 256)         38745344  
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 300, 256)         280864    
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 256)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 256)               0     

# 모델 학습

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=256, callbacks=callbacks,epochs=50, validation_data=(x_val, y_val))

print("테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

Epoch 1/50
55/55 [==============================] - 4s 42ms/step - loss: 3.0007 - accuracy: 0.0855 - val_loss: 2.6530 - val_accuracy: 0.2700 - lr: 0.0010
Epoch 2/50
55/55 [==============================] - 2s 38ms/step - loss: 1.4547 - accuracy: 0.5545 - val_loss: 0.6715 - val_accuracy: 0.8100 - lr: 0.0010
Epoch 3/50
23/55 [===========>..................] - ETA: 1s - loss: 0.4223 - accuracy: 0.8818

In [14]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_45.h5', custom_objects=custom_objects)
model.evaluate(x_test, y_test)

118/118 [==============================] - 1s 7ms/step - loss: 0.8070 - accuracy: 0.8715


[0.8069727420806885, 0.8714816570281982]